<a href="https://colab.research.google.com/github/patelarth/pegasus-onnews-ads/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#if the notebook is colab 
!pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install sentencepiece
!pip install transformers
!pip install rouge-score

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |█████████████▌                  | 834.1 MB 1.5 MB/s eta 0:12:48tcmalloc: large alloc 1147494400 bytes == 0x55f8e3820000 @  0x7f08ef0e0615 0x55f8e1ccd4cc 0x55f8e1dad47a 0x55f8e1cd02ed 0x55f8e1dc1e1d 0x55f8e1d43e99 0x55f8e1d3e9ee 0x55f8e1cd1bda 0x55f8e1d43d00 0x55f8e1d3e9ee 0x55f8e1cd1bda 0x55f8e1d40737 0x55f8e1dc2c66 0x55f8e1d3fdaf 0x55f8e1dc2c66 0x55f8e1d3fdaf 0x55f8e1dc2c66 0x55f8e1d3fdaf 0x55f8e1cd2039 0x55f8e1d15409 0x55f8e1cd0c52 0x55f8e1d43c25 0x55f8e1d3e9ee 0x55f8e1cd1bda 0x55f8e1d40737 0x55f8e1d3e9ee 0x55f8e1cd1bda 0x55f8e1d3f915 0x55f8e1cd1afa 0x55f8e1d3fc0d 0x55f8e1d3e9ee
     |█████████████████               | 1055.7 MB 1.3 MB/s eta 0:11:31tcmalloc: large alloc 1434370048 bytes == 0x55f927e76000 @  0x7f08ef0e0615 0x55f8e1ccd4cc 0x55f8e1dad47a 0x55f8e1cd02ed 0x55f8e1dc1e1d 0x55f8e1d43e99 0x55f8e1d3e9ee 0x55f8e1cd1bda 0x55f8e1d43d00 0x55f8e1d3e9ee 0x55f8e1cd1bda 0x55f8e1d40737 0x55f8e1dc2c66 0x55f8e

In [2]:
links_list = ['https://7news.com.au/politics/world-politics/emmanuel-macron-tells-scott-morrison-trust-was-broken-after-aukus-submarine-deal-c-4358673',
              'https://7news.com.au/sport/afl/afl-eagle-jones-charged-with-drink-driving-c-4359074',
              'https://7news.com.au/politics/gladys-berejiklian/berejiklian-next-to-face-grilling-by-icac-c-4360136',
              'https://7news.com.au/sunrise/on-the-show/covid-19-rapid-antigen-tests-now-available-at-supermarkets-and-pharmacies-c-4382586',
              'https://7news.com.au/news/missing-person/nsw-police-issue-alert-for-kathleen-riethmuller-after-the-lane-cove-woman-mysteriously-vanished-c-4378401',
              'https://7news.com.au/lifestyle/health-wellbeing/new-covid-19-cases-in-nsw-drop-to-low-not-seen-since-july-c-4377068',
              'https://7news.com.au/politics/joyce-details-some-of-nationals-2050-deal-c-4377609',
              'https://7news.com.au/news/vic/kill-the-bill-thousands-of-protesters-take-to-melbourne-streets-over-covid-19-pandemic-laws-c-4372996',
              'https://7news.com.au/entertainment/tv/armorer-on-the-set-of-rust-says-she-has-no-idea-where-deadly-ammunition-that-killed-halyna-hutchins-came-from-c-4371273',
              'https://7news.com.au/news/world/just-two-per-cent-of-elon-musks-wealth-could-solve-world-hunger-according-to-un-food-scarcity-organisation-c-4345980']

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd

class scrap:

    src = []
    summary = []
    df = pd.DataFrame()

    def __init__(self):
        ''' constructor method for an class'''
        self.src = []
        self.summary = []


    def preprocess_data(self,text):
        '''it will preprocess the text which contain the number and any other special charaters and return the preprocess text'''
        text = re.sub(r'\s+',' ',text)  # remove the special char. from the text
        text = re.sub(r'\d','',text)
        return text


    def scrap_via_bs4(self,links):
        'input link, it will scrap the data from the link and store in variable'
        for link in links:
            url = link
            html = urlopen(url) # make an connection to the link and apply bs4
            soup = BeautifulSoup(html, 'html.parser')
            text = ""
            title = ""
            for paragraph in soup.find_all('p'):
                text += paragraph.get_text()
                process_text = self.preprocess_data(text)
                process_text = '.'.join(process_text.split('.')) + '.'
            for paragraph in soup.find_all('h1'):
                title += paragraph.get_text()
                process_title = self.preprocess_data(text)
                process_title = '.'.join(process_title.split('.')) + '.'
            self.src.append(text)
            self.summary.append (title)

    def genrate_data_frame(self):
        ''' this function will genrate the data frame of the form the src and summary data.
         first have to genrate scrape data from the url'''
        self.df = pd.DataFrame({'text': self.src,
                                'summary': self.summary})
        return self.df

    def create_csv(self,name):
        ''' this function require file name with and extension .csv (other excel format are accepted) with the file path and create file of the scrapped dataset'''
        self.df.to_csv(name)
        print('File created')

In [4]:
object = scrap()
object.scrap_via_bs4(links_list)
df = object.genrate_data_frame()

In [5]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
from rouge_score import rouge_scorer

In [6]:
def model_train_pegasus(datasetname = 'google/pegasus-cnn_dailymail'):
  dataset_name = datasetname 
  # check if cude is availabel if yes then use cuda for processing
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  tokenizer_pegasus = PegasusTokenizer.from_pretrained(dataset_name)
  model = PegasusForConditionalGeneration.from_pretrained(dataset_name).to(device)
  return tokenizer_pegasus,model

tokenizer,model = model_train_pegasus()

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [7]:
def model_abstractive_summary_gen(src_text,tokenizer,model):
  batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt") #gap sentence gentartion and encoding 
  translated = model.generate(**batch)
  abstrct_sumtext = tokenizer.batch_decode(translated, skip_special_tokens=True) # decode
  return abstrct_sumtext